## Parametrization

In [1]:
# remember to run nbdev_build_lib first! 

# !nbdev_build_lib
# import compmemlearn
# import importlib
# import repetition_cmr
# importlib.reload(repetition_cmr.analyses)
# from repetition_cmr.analyses import recall_probability_by_lag, spc, lag_crp, repetition_contiguity

In [2]:
# | code-summary: code -- load dependencies and data and select parameters

from compmemlearn.model_analysis import matrix_heatmap, icmr_memory_heatmap 
from compmemlearn.model_analysis import connectivity_by_lag, mfc_heatmap
from compmemlearn.analyses import flex_mixed_pfr
from compmemlearn.fitting import generate_objective_function, apply_and_concatenate
from compmemlearn.datasets import simulate_df, simulate_array_from_presentations
from repetition_cmr.model_evaluation import repetition_connectivity_by_lag, latent_mfc_mcf_mff, mixed_connectivity_by_lag
from repetition_cmr.analyses import recall_probability_by_lag, spc, lag_crp, repetition_contiguity
from scipy.optimize import differential_evolution
from numba.typed import List, Dict
from numba.core import types
from numba import njit
from psifr import fr
import pandas as pd
import numpy as np
from numpy import matlib
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import importlib

lb = np.finfo(float).eps
ub = 1 - np.finfo(float).eps

section_tag = "InstanceCMR"

data_path = "../../data/LohnasKahana2014.csv"
data_query = "condition == 4"

model_paths = ["compmemlearn.models.Instance_CMR"]

model_names = ["InstanceCMR"]

parameter_ranges = {
    #'feature_drift_rate': np.arange(.001, 1, .1),
    'context_sensitivity': np.arange(1.0, 10.0, 1),
}

free_parameters = [
        [
        "encoding_drift_rate",
        "start_drift_rate",
        "recall_drift_rate",
        "shared_support",
        "item_support",
        "learning_rate",
        "primacy_scale",
        "primacy_decay",
        "stop_probability_scale",
        "stop_probability_growth",
        #'choice_sensitivity',
            "context_sensitivity",
        #    "feature_sensitivity",
        "delay_drift_rate",
    ]
]

bounds = [
        [
        [lb, ub],
        [lb, ub],
        [lb, ub],
        [lb, ub],
        [lb, ub],
        [lb, ub],
        [lb, 100],
        [lb, 100],
        [lb, ub],
        [lb, 10],
        [lb, 10],
        [lb, ub],
    ],
]

fixed_parameters = [
    {"choice_sensitivity": 1.0, "feature_sensitivity": 1.0},
]

fit_result = [

]

benchmark_experiment_count = 100
parameter_shifting_experiment_count = 100

In [3]:
#| code-summary: code -- test that specified parameters are valid
#| output: false

# load dataset and extract key metadata
events = pd.read_csv(data_path).query(data_query)

# retrieve important metadata
list_length = int(np.max(events.input))
presentations_df = events.pivot_table(index=['subject', 'list'], columns='input', values='item')
presentations = presentations_df.to_numpy(na_value=0).astype('int32')
trials_df = events.pivot_table(index=['subject', 'list'], columns='output', values='item')
trials = trials_df.to_numpy(na_value=0).astype('int32')

# import models from specified source
models = []
for i in range(len(model_paths)):
    module_name, model_name = model_paths[i].rsplit('.',1) #TODO: add handling for case with no `.`
    module = importlib.import_module(module_name)
    models.append(getattr(module, model_name))

# make sure model initializes with provided parameters and boundaries
for model_index, model_class in enumerate(models):

    @njit(fastmath=True, nogil=True)
    def init_model(item_count, presentation_count, parameters):
        return model_class(item_count, presentation_count, parameters)

    cost_function = generate_objective_function(
        events,
        init_model,
        fixed_parameters[model_index],
        free_parameters[model_index],
    )
    
    for boundary_index in range(2):
        x = np.array([each[boundary_index] for each in bounds[model_index]])

        # parameter configuration
        parameters = Dict.empty(key_type=types.unicode_type, value_type=types.float64)
        for name, value in fixed_parameters[model_index].items():
            parameters[name] = value
        for i in range(len(free_parameters[model_index])):
                parameters[free_parameters[model_index][i]] = x[i]

        model = init_model(20, 20, parameters)
        model.experience(model.items)
        model.free_recall()

        cost_function(x)

## Fitting

In [4]:
#| code-summary: code -- 1) fit models to entire dataset
#| output: false

if len(fit_result) != len(models):
    for model_index, model_class in enumerate(models):

        @njit(fastmath=True, nogil=True)
        def init_model(item_count, presentation_count, parameters):
            return model_class(item_count, presentation_count, parameters)

        cost_function = generate_objective_function(
                events,
                init_model,
                fixed_parameters[model_index],
                free_parameters[model_index],
            )

        fit_result.append(
            differential_evolution(
                cost_function, bounds[model_index], disp=True
            ))
        print(fit_result[-1])
        fit_result[-1] = fit_result[-1].x

```
     fun: 18446.993195825322
     jac: array([ -6.66150296, -10.20052878,  -7.97881508, -80.34112398,
         1.26419764,  13.82286428,  -6.76809581,   0.        ,
         1.5275873 ,   3.66599125,   0.        ,   4.50781952])
 message: 'Optimization terminated successfully.'
    nfev: 9975
     nit: 36
 success: True
       x: array([8.62442206e-01, 6.22194736e-02, 9.71688696e-01, 6.22838475e-04,
       1.13888859e-01, 9.82938071e-02, 3.28203304e+00, 5.89254366e+01,
       2.08236313e-02, 1.09607676e-01, 4.50266111e-01, 9.95517136e-01])
```

## Memory Simulation

In [ ]:
fitted_parameters = []
for model_index, model_class in enumerate(models):

    fitted_parameters.append(Dict.empty(
        key_type=types.unicode_type, value_type=types.float64
    ))
    for i in range(len(fit_result[model_index])):
        fitted_parameters[-1][free_parameters[model_index][i]] = fit_result[model_index][i]
    for key in fixed_parameters[model_index]:
        fitted_parameters[-1][key] = fixed_parameters[model_index][key]

    model = model_class(list_length, list_length, fitted_parameters[-1])
    simulate_df(model, 1)

    icmr_memory_heatmap(model, just_experimental=True, just_context=False)

    item_count = np.max(presentations[0])+1
    model = model_class(item_count, list_length, fitted_parameters[-1])
    model.experience(model.items[presentations[0]])
    icmr_memory_heatmap(model, just_experimental=False, just_context=False)
    print(presentations[0])

## Benchmark Evaluation

In [ ]:
for model_index, model_class in enumerate(models):
    model_name = model_names[model_index]

    @njit(fastmath=True, nogil=True)
    def init_model(item_count, presentation_count, parameters):
        return model_class(item_count, presentation_count, parameters)
    
    # first_recalls = np.zeros(len(presentations), dtype=int)
    # for i in range(len(presentations)):
    #     first_recalls[i] = presentations[i][trials[i, 0]-1]+1
        
    sim_trials = simulate_array_from_presentations(
        init_model, fitted_parameters[model_index], presentations, benchmark_experiment_count)
        #init_model, sim_fitted_parameters, presentations, benchmark_experiment_count, first_recalls)
    sim_presentations = np.matlib.repmat(presentations, benchmark_experiment_count, 1)

    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(20, 20), sharey='row')

    # spc
    data_spc= spc(trials, presentations)
    sim_spc = spc(sim_trials, sim_presentations)
    axes[0, 0].plot(np.arange(len(data_spc)), data_spc, label='Data')
    axes[0, 0].plot(np.arange(len(sim_spc)), sim_spc, label=model_name)
    axes[0, 0].set_title('SPC')

    # pfr
    data_pfr = flex_mixed_pfr(trials, presentations)
    sim_pfr = flex_mixed_pfr(sim_trials, sim_presentations)
    axes[1, 1].plot(np.arange(len(data_pfr)), data_pfr, label='Data')
    axes[1, 1].plot(np.arange(len(sim_pfr)), sim_pfr, label=model_name)
    axes[1, 1].set_title('PFR')

    # crp
    lag_range = len(presentations[0])-1
    data_crp= lag_crp(trials, presentations)
    data_crp[lag_range] = np.nan
    sim_crp = lag_crp(sim_trials, sim_presentations)
    sim_crp[lag_range] = np.nan
    axes[1, 0].plot(np.arange(len(data_crp)), data_crp, label='Data')
    axes[1, 0].plot(np.arange(len(sim_crp)), sim_crp, label=model_name)
    axes[1, 0].set_xticks(np.arange(0, len(data_crp), 4))
    axes[1, 0].set_xticklabels(np.arange(0, len(data_crp), 4) - lag_range)
    axes[1, 0].set_title('CRP')

    # rpl
    data_rpl = recall_probability_by_lag(trials, presentations, max_lag=8)
    binned = np.zeros(5)
    binned[0] = data_rpl[0]
    binned[1] = data_rpl[1]
    binned[2] = (data_rpl[2] + data_rpl[3])/2
    binned[3] = (data_rpl[4] + data_rpl[5] + data_rpl[6])/3
    binned[4] = (data_rpl[7] + data_rpl[8] + data_rpl[9])/3
    data_rpl = binned.copy()

    sim_rpl = recall_probability_by_lag(sim_trials, sim_presentations, max_lag=8)
    binned = np.zeros(5)
    binned[0] = sim_rpl[0]
    binned[1] = sim_rpl[1]
    binned[2] = (sim_rpl[2] + sim_rpl[3])/2
    binned[3] = (sim_rpl[4] + sim_rpl[5] + sim_rpl[6])/3
    binned[4] = (sim_rpl[7] + sim_rpl[8] + sim_rpl[9])/3
    sim_rpl = binned.copy()

    axes[0, 1].plot(np.arange(len(data_rpl)), data_rpl, label='Data')
    axes[0, 1].plot(np.arange(len(sim_rpl)), sim_rpl, label=model_name)
    axes[0, 1].set_title('Recall Probability by Lag')

    # rep_ contiguity (data)
    data_rep_crp = repetition_contiguity(
        trials, presentations, 6, 2)
    data_rep_crp[:, lag_range] = np.nan
    axes[2, 0].plot(np.arange(7), data_rep_crp[0][lag_range-3:lag_range+4], label='First Presentation')
    axes[2, 0].plot(np.arange(7), data_rep_crp[1][lag_range-3:lag_range+4], label='Second Presentation')
    axes[2, 0].set_xticks(np.arange(7))
    axes[2, 0].set_xticklabels(np.arange(7) - 3)
    axes[2, 0].set_title('Repetition Contiguity -- Data')

    sim_rep_crp = repetition_contiguity(
        sim_trials, sim_presentations, 6, 2)
    sim_rep_crp[:, lag_range] = np.nan
    axes[2, 1].plot(np.arange(7), sim_rep_crp[0][lag_range-3:lag_range+4], label='First Presentation')
    axes[2, 1].plot(np.arange(7), sim_rep_crp[1][lag_range-3:lag_range+4], label='Second Presentation')
    axes[2, 1].set_xticks(np.arange(7))
    axes[2, 1].set_xticklabels(np.arange(7) - 3)
    axes[2, 1].set_title('Repetition Contiguity -- ' + model_name)

    axes[0, 1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    axes[2, 1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10), sharey='row')

    # repetition contiguity (data)
    data_rep_crp = repetition_contiguity(
        trials, presentations, 6, 2)
    data_rep_crp[:, lag_range] = np.nan
    sim_rep_crp = repetition_contiguity(
        sim_trials, sim_presentations, 6, 2)
    sim_rep_crp[:, lag_range] = np.nan

    axes.plot(np.arange(7), data_rep_crp[0][lag_range-3:lag_range+4]-data_rep_crp[1][lag_range-3:lag_range+4], label='Data')
    axes.plot(np.arange(7), sim_rep_crp[0][lag_range-3:lag_range+4]-sim_rep_crp[1][lag_range-3:lag_range+4], label=model_name)
    axes.set_xticks(np.arange(7))
    axes.set_xticklabels(np.arange(7) - 3)
    axes.set_title('Positional Contiguity Difference')

    axes.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

## Memory Connectivity

In [ ]:
global_lag_range = len(presentations[0]) - 1

for model_index, model_class in enumerate(models):

    mfc_alternative_connectivities = np.zeros((2, global_lag_range * 2 + 1))
    mcf_alternative_connectivities = np.zeros((2, global_lag_range * 2 + 1))
    mff_alternative_connectivities = np.zeros((2, global_lag_range * 2 + 1))

    # loop through presentations
    for trial_index, presentation in enumerate(presentations):

        # simulate list study
        item_count = np.max(presentation)+1
        model = model_class(item_count, len(presentation), fitted_parameters[model_index])
        model.experience(model.items[presentation])

        # extract item connections
        try:
            latent_mfc, mcf_connections, mff_connections = latent_mfc_mcf_mff(model, model.recall_items)
        except:
            latent_mfc, mcf_connections, mff_connections = latent_mfc_mcf_mff(model, model.items)
        mfc_connections = latent_mfc[:, 1:-1]

        # track alternative connectivity
        mfc_alternative_connectivities += repetition_connectivity_by_lag(mfc_connections, presentation)
        mcf_alternative_connectivities += repetition_connectivity_by_lag(mcf_connections, presentation)
        mff_alternative_connectivities += repetition_connectivity_by_lag(mff_connections, presentation)

    # reduce sum to mean
    mfc_alternative_connectivity = mfc_alternative_connectivities / (trial_index+1)
    mcf_alternative_connectivity = mcf_alternative_connectivities / (trial_index+1)
    mff_alternative_connectivity = mff_alternative_connectivities / (trial_index+1)
    mfc_alternative_connectivity[:, global_lag_range] = np.nan
    mcf_alternative_connectivity[:, global_lag_range] = np.nan
    mff_alternative_connectivity[:, global_lag_range] = np.nan

    # focus on +/- 3 lags
    plotting_lag_range = 3
    mfc_alternative_connectivity = mfc_alternative_connectivity[
        :, global_lag_range-plotting_lag_range:global_lag_range+plotting_lag_range+1]
    mcf_alternative_connectivity = mcf_alternative_connectivity[
        :, global_lag_range-plotting_lag_range:global_lag_range+plotting_lag_range+1]
    mff_alternative_connectivity = mff_alternative_connectivity[
        :, global_lag_range-plotting_lag_range:global_lag_range+plotting_lag_range+1]

    # plot results
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 5), sharey=False)
    axes[0].plot(np.arange(len(mcf_alternative_connectivity[0])), mcf_alternative_connectivity[0], label='First Presentation')
    axes[0].plot(np.arange(len(mcf_alternative_connectivity[1])), mcf_alternative_connectivity[1], label='Second Presentation')
    axes[0].set_xticks(np.arange(0, plotting_lag_range * 2 + 1, 2))
    axes[0].set_xticklabels(np.arange(0, plotting_lag_range * 2 + 1, 2) - plotting_lag_range)
    axes[0].set_title('MCF')

    # same for MFC
    axes[1].plot(np.arange(len(mfc_alternative_connectivity[0])), mfc_alternative_connectivity[0], label='First Presentation')
    axes[1].plot(np.arange(len(mfc_alternative_connectivity[1])), mfc_alternative_connectivity[1], label='Second Presentation')
    axes[1].set_xticks(np.arange(0, plotting_lag_range * 2 + 1, 2))
    axes[1].set_xticklabels(np.arange(0, plotting_lag_range * 2 + 1, 2) - plotting_lag_range)
    axes[1].set_title('MFC')

    # same for MFF
    axes[2].plot(np.arange(len(mff_alternative_connectivity[0])), mff_alternative_connectivity[0], label='First Presentation')
    axes[2].plot(np.arange(len(mff_alternative_connectivity[1])), mff_alternative_connectivity[1], label='Second Presentation')
    axes[2].set_xticks(np.arange(0, plotting_lag_range * 2 + 1, 2))
    axes[2].set_xticklabels(np.arange(0, plotting_lag_range * 2 + 1, 2) - plotting_lag_range)
    axes[2].set_title('MFF')


## Parameter Shifting

In [ ]:
global_lag_range = list_length-1
crp_max_lag = 13
rep_crp_max_lag = 3
sim_presentations = np.matlib.repmat(presentations, parameter_shifting_experiment_count, 1)

for model_index, model_class in enumerate(models):

    @njit(fastmath=True, nogil=True)
    def init_model(item_count, presentation_count, parameters):
        return model_class(item_count, presentation_count, parameters)

    for varied_parameter in parameter_ranges.keys():

        print(varied_parameter, fitted_parameters[model_index][varied_parameter])
        print(parameter_ranges[varied_parameter])
        spcs = []
        pfrs = []
        rpls = []
        crps = []
        rep_crps = []
        lag_mcfs = []
        lag_mfcs = []
        lag_mffs = []
        rep_lag_mcfs = []
        rep_lag_mfcs = []
        rep_lag_mffs = []

        # loop through parameter values
        for parameter_value in tqdm(parameter_ranges[varied_parameter]):

            # simulate data with this parameter value modified
            model_parameters = fitted_parameters[model_index].copy()
            model_parameters[varied_parameter] = parameter_value
            sim_trials = simulate_array_from_presentations(
                init_model, model_parameters, presentations, parameter_shifting_experiment_count)

            # spc
            sim_spc = spc(sim_trials, sim_presentations)
            sim_spc = pd.DataFrame(
                    {'Study Position': np.arange(len(sim_spc)), 'Recall Rate': sim_spc, varied_parameter: parameter_value})
            spcs.append(sim_spc)

            # pfr
            sim_pfr = flex_mixed_pfr(sim_trials, sim_presentations)
            sim_pfr = pd.DataFrame(
                {'Study Position': np.arange(len(sim_pfr)), 'First Recall Rate': sim_pfr, varied_parameter: parameter_value})
            pfrs.append(sim_pfr)

            # rpl
            rpl = recall_probability_by_lag(
                sim_trials, sim_presentations, max_lag=8)
            binned = np.zeros(5)
            binned[0] = rpl[0]
            binned[1] = rpl[1]
            binned[2] = (rpl[2] + rpl[3])/2
            binned[3] = (rpl[4] + rpl[5] + rpl[6])/3
            binned[4] = (rpl[7] + rpl[8] + rpl[9])/3
            rpl = binned.copy()
            rpl = pd.DataFrame(
                {'Spacing': ['N/A', '0', '1-2', '3-5', '6-8'], 'Recall Rate': rpl, varied_parameter: parameter_value})
            rpls.append(rpl)

            # crp
            sim_crp = lag_crp(sim_trials, sim_presentations)
            sim_crp[global_lag_range] = np.nan
            sim_crp = pd.DataFrame(
                {'Lag': np.arange(crp_max_lag*2 + 1, dtype=int)-crp_max_lag, 
                'Recall Rate': sim_crp[list_length-crp_max_lag-1:list_length+crp_max_lag], 
                varied_parameter: parameter_value})
            crps.append(sim_crp)

            # rep crp
            rep_crp = repetition_contiguity(
                sim_trials, sim_presentations, 6, 2)
            rep_crp[:, global_lag_range] = np.nan
            rep_crp = pd.DataFrame({
                    "Lag": np.arange(rep_crp_max_lag * 2 + 1, dtype=int) - rep_crp_max_lag,
                    "Differential Recall Rate": rep_crp[0, list_length-rep_crp_max_lag-1:list_length + rep_crp_max_lag]
                        - rep_crp[1, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag],
                    varied_parameter: parameter_value,
                })
            rep_crps.append(rep_crp)

            # lag-connectivity requires more involved computations...
            total_lag_mfc = np.zeros((global_lag_range * 2 + 1))
            total_lag_mcf = np.zeros((global_lag_range * 2 + 1))
            total_lag_mff = np.zeros((global_lag_range * 2 + 1))
            total_rep_lag_mfc = np.zeros((2, global_lag_range * 2 + 1))
            total_rep_lag_mcf = np.zeros((2, global_lag_range * 2 + 1))
            total_rep_lag_mff = np.zeros((2, global_lag_range * 2 + 1))

            # loop through presentations
            for trial_index, presentation in enumerate(presentations):

                # simulate list study
                item_count = np.max(presentation)+1
                model = model_class(item_count, len(presentation), model_parameters)
                model.experience(model.items[presentation])

                # extract item connections
                try:
                    latent_mfc, mcf_connections, mff_connections = latent_mfc_mcf_mff(model, model.recall_items)
                except:
                    latent_mfc, mcf_connections, mff_connections = latent_mfc_mcf_mff(model, model.items)
                mfc_connections = latent_mfc[:, 1:-1]

                # track repetition connectivity
                total_rep_lag_mfc += repetition_connectivity_by_lag(mfc_connections, presentation)
                total_rep_lag_mcf += repetition_connectivity_by_lag(mcf_connections, presentation)
                total_rep_lag_mff += repetition_connectivity_by_lag(mff_connections, presentation)

                # and overall connectivity
                total_lag_mfc += mixed_connectivity_by_lag(mfc_connections, presentation)
                total_lag_mcf += mixed_connectivity_by_lag(mcf_connections, presentation)
                total_lag_mff += mixed_connectivity_by_lag(mff_connections, presentation)

            # reduce sum to mean
            # repetition connectivity
            rep_lag_mfc = total_rep_lag_mfc / (trial_index+1)
            rep_lag_mcf = total_rep_lag_mcf / (trial_index+1)
            rep_lag_mff = total_rep_lag_mff / (trial_index+1)
            rep_lag_mfc[:, global_lag_range] = np.nan
            rep_lag_mcf[:, global_lag_range] = np.nan
            rep_lag_mff[:, global_lag_range] = np.nan

            # overall connectivity
            lag_mfc = total_lag_mfc / (trial_index+1)
            lag_mcf = total_lag_mcf / (trial_index+1)
            lag_mff = total_lag_mff / (trial_index+1)
            lag_mfc[global_lag_range] = np.nan
            lag_mcf[global_lag_range] = np.nan
            lag_mff[global_lag_range] = np.nan

            # aggregate for dataframe
            rep_lag_mfc = pd.DataFrame(
                {'Lag': np.arange(rep_crp_max_lag*2+1, dtype=int)-rep_crp_max_lag,
                'Differential Recall Rate': rep_lag_mfc[0, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag]
                - rep_lag_mfc[1, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag],
                varied_parameter: parameter_value})
            rep_lag_mcf = pd.DataFrame(
                {'Lag': np.arange(rep_crp_max_lag*2+1, dtype=int)-rep_crp_max_lag,
                'Differential Recall Rate': rep_lag_mcf[0, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag]
                - rep_lag_mcf[1, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag],
                varied_parameter: parameter_value})
            rep_lag_mff = pd.DataFrame(
                {'Lag': np.arange(rep_crp_max_lag*2+1, dtype=int)-rep_crp_max_lag,
                'Differential Recall Rate': rep_lag_mff[0, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag]
                - rep_lag_mff[1, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag],
                varied_parameter: parameter_value})

            lag_mfc = pd.DataFrame(
                {'Lag': np.arange(crp_max_lag*2+1, dtype=int)-crp_max_lag,
                'Recall Rate': lag_mfc[list_length-crp_max_lag-1:list_length+crp_max_lag],
                varied_parameter: parameter_value})
            lag_mcf = pd.DataFrame(
                {'Lag': np.arange(crp_max_lag*2+1, dtype=int)-crp_max_lag,
                'Recall Rate': lag_mcf[list_length-crp_max_lag-1:list_length+crp_max_lag],
                varied_parameter: parameter_value})
            lag_mff = pd.DataFrame(
                {'Lag': np.arange(crp_max_lag*2+1, dtype=int)-crp_max_lag,
                'Recall Rate': lag_mff[list_length-crp_max_lag-1:list_length+crp_max_lag],
                varied_parameter: parameter_value})

            rep_lag_mcfs.append(rep_lag_mcf)
            rep_lag_mffs.append(rep_lag_mff)
            rep_lag_mfcs.append(rep_lag_mfc)
            lag_mcfs.append(lag_mcf)
            lag_mffs.append(lag_mff)
            lag_mfcs.append(lag_mfc)
        
        # aggregate fitting results
        full_spc = pd.concat(spcs).reset_index()
        full_pfr = pd.concat(pfrs).reset_index()
        full_crp = pd.concat(crps).reset_index()
        rpl = pd.concat(rpls).reset_index()
        rep_crp = pd.concat(rep_crps).reset_index()
        lag_mcf = pd.concat(lag_mcfs).reset_index()
        lag_mff = pd.concat(lag_mffs).reset_index()
        lag_mfc = pd.concat(lag_mfcs).reset_index()
        rep_lag_mcf = pd.concat(rep_lag_mcfs).reset_index()
        rep_lag_mff = pd.concat(rep_lag_mffs).reset_index()
        rep_lag_mfc = pd.concat(rep_lag_mfcs).reset_index()

        # plot results
        sns.set(style='darkgrid')
        fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(25, 25), sharey=False)

        # spc
        sns.lineplot(ax=axes[0, 0], data=full_spc, x='Study Position', y='Recall Rate', hue=varied_parameter, ci=None)
        axes[0, 0].set_title('SPC')

        # pfr
        sns.lineplot(ax=axes[0, 1], data=full_pfr, x='Study Position', y='First Recall Rate', hue=varied_parameter, ci=None, legend=False)
        axes[0, 1].set_title('PFR')

        # rpl (should be rps)
        sns.lineplot(ax=axes[0, 2], data=rpl, x='Spacing', y='Recall Rate', hue=varied_parameter, ci=None)
        axes[0, 2].set_title('Recall Probability by Spacing')

        # crp
        filt_neg = f'{-crp_max_lag} <= Lag < 0'
        filt_pos = f'0 < Lag <= {crp_max_lag}'
        sns.lineplot(ax=axes[1, 0], data=full_crp.query(filt_neg), x='Lag', y='Recall Rate', 
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[1, 0], data=full_crp.query(filt_pos), x='Lag', y='Recall Rate', 
            hue=varied_parameter, ci=None, legend=False)
        axes[1, 0].set_title('CRP')

        # rep_crp
        sns.lineplot(ax=axes[1, 1], data=rep_crp.query(filt_neg), x='Lag', y="Differential Recall Rate", 
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[1, 1], data=rep_crp.query(filt_pos), x='Lag', y="Differential Recall Rate", 
            hue=varied_parameter, ci=None, legend=False)
        axes[1, 1].set_title('Repetition Lag-Contiguity Difference')

        # lag_mcf
        sns.lineplot(ax=axes[2, 0], data=lag_mcf.query(filt_neg), x='Lag', y='Recall Rate',
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[2, 0], data=lag_mcf.query(filt_pos), x='Lag', y='Recall Rate',
            hue=varied_parameter, ci=None, legend=False)

        # lag_mfc
        sns.lineplot(ax=axes[2, 1], data=lag_mfc.query(filt_neg), x='Lag', y='Recall Rate',
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[2, 1], data=lag_mfc.query(filt_pos), x='Lag', y='Recall Rate',
            hue=varied_parameter, ci=None, legend=False)

        # lag_mff
        sns.lineplot(ax=axes[2, 2], data=lag_mff.query(filt_neg), x='Lag', y='Recall Rate',
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[2, 2], data=lag_mff.query(filt_pos), x='Lag', y='Recall Rate',
            hue=varied_parameter, ci=None, legend=False)

        # rep_lag_mcf
        sns.lineplot(ax=axes[3, 0], data=rep_lag_mcf.query(filt_neg), x='Lag', y='Differential Recall Rate',
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[3, 0], data=rep_lag_mcf.query(filt_pos), x='Lag', y='Differential Recall Rate',
            hue=varied_parameter, ci=None, legend=False)

        # rep_lag_mfc
        sns.lineplot(ax=axes[3, 1], data=rep_lag_mfc.query(filt_neg), x='Lag', y='Differential Recall Rate',
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[3, 1], data=rep_lag_mfc.query(filt_pos), x='Lag', y='Differential Recall Rate',
            hue=varied_parameter, ci=None, legend=False)

        # rep_lag_mff
        sns.lineplot(ax=axes[3, 2], data=rep_lag_mff.query(filt_neg), x='Lag', y='Differential Recall Rate',
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[3, 2], data=rep_lag_mff.query(filt_pos), x='Lag', y='Differential Recall Rate',
            hue=varied_parameter, ci=None, legend=False)

        # render result
        plt.show()

In [ ]:
for model_index, model_class in enumerate(models):

    @njit(fastmath=True, nogil=True)
    def init_model(item_count, presentation_count, parameters):
        return model_class(item_count, presentation_count, parameters)
        
    for varied_parameter in parameter_ranges.keys():

        print(varied_parameter, fitted_parameters[model_index][varied_parameter])
        rep_crps = []
        rep_lag_mcfs = []
        rep_lag_mfcs = []
        rep_lag_mffs = []

        # loop through parameter values
        for parameter_value in tqdm(parameter_ranges[varied_parameter]):

            # simulate data with this parameter value modified
            model_parameters = fitted_parameters[model_index].copy()
            model_parameters[varied_parameter] = parameter_value
            sim_trials = simulate_array_from_presentations(
                init_model, model_parameters, presentations, parameter_shifting_experiment_count)
            

            # rep crp
            rep_crp = repetition_contiguity(sim_trials, sim_presentations, 6, 2)
            rep_crp[:, global_lag_range] = np.nan
            rep_crp = pd.DataFrame({
                    "Lag": (np.arange(rep_crp_max_lag*2+1, dtype=int)-rep_crp_max_lag).tolist() * 2,
                    "Recall Rate": rep_crp[0, list_length-rep_crp_max_lag-1:list_length + rep_crp_max_lag].tolist()
                        + rep_crp[1, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag].tolist(),
                    varied_parameter: parameter_value,
                    "Position": ['First'] * (rep_crp_max_lag*2+1) + ['Second'] * (rep_crp_max_lag*2+1)
                })
            rep_crps.append(rep_crp)

            # lag-connectivity requires more involved computations...
            total_rep_lag_mfc = np.zeros((2, global_lag_range * 2 + 1))
            total_rep_lag_mcf = np.zeros((2, global_lag_range * 2 + 1))
            total_rep_lag_mff = np.zeros((2, global_lag_range * 2 + 1))

            # loop through presentations
            for trial_index, presentation in enumerate(presentations):

                # simulate list study
                item_count = np.max(presentation)+1
                model = model_class(item_count, len(presentation), model_parameters)
                model.experience(model.items[presentation])

                # extract item connections
                try:
                    latent_mfc, mcf_connections, mff_connections = latent_mfc_mcf_mff(model, model.recall_items)
                except:
                    latent_mfc, mcf_connections, mff_connections = latent_mfc_mcf_mff(model, model.items)
                mfc_connections = latent_mfc[:, 1:-1]

                # track repetition connectivity
                total_rep_lag_mfc += repetition_connectivity_by_lag(mfc_connections, presentation)
                total_rep_lag_mcf += repetition_connectivity_by_lag(mcf_connections, presentation)
                total_rep_lag_mff += repetition_connectivity_by_lag(mff_connections, presentation)

            # reduce sum to mean
            # repetition connectivity
            rep_lag_mfc = total_rep_lag_mfc / (trial_index+1)
            rep_lag_mcf = total_rep_lag_mcf / (trial_index+1)
            rep_lag_mff = total_rep_lag_mff / (trial_index+1)
            rep_lag_mfc[:, global_lag_range] = np.nan
            rep_lag_mcf[:, global_lag_range] = np.nan
            rep_lag_mff[:, global_lag_range] = np.nan

            # aggregate for dataframe
            rep_lag_mfc = pd.DataFrame(
                {'Lag': (np.arange(rep_crp_max_lag*2+1, dtype=int)-rep_crp_max_lag).tolist() * 2,
                'Recall Rate': rep_lag_mfc[0, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag].tolist()
                + rep_lag_mfc[1, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag].tolist(),
                varied_parameter: parameter_value,
                "Position": ['First'] * (rep_crp_max_lag*2+1) + ['Second'] * (rep_crp_max_lag*2+1)})
            rep_lag_mcf = pd.DataFrame(
                {'Lag': (np.arange(rep_crp_max_lag*2+1, dtype=int)-rep_crp_max_lag).tolist() * 2,
                'Recall Rate': rep_lag_mcf[0, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag].tolist()
                + rep_lag_mcf[1, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag].tolist(),
                varied_parameter: parameter_value,
                "Position": ['First'] * (rep_crp_max_lag*2+1) + ['Second'] * (rep_crp_max_lag*2+1)})
            rep_lag_mff = pd.DataFrame(
                {'Lag': (np.arange(rep_crp_max_lag*2+1, dtype=int)-rep_crp_max_lag).tolist() * 2,
                'Recall Rate': rep_lag_mff[0, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag].tolist()
                + rep_lag_mff[1, list_length-rep_crp_max_lag-1:list_length+rep_crp_max_lag].tolist(),
                varied_parameter: parameter_value,
                "Position": ['First'] * (rep_crp_max_lag*2+1) + ['Second'] * (rep_crp_max_lag*2+1)})

            rep_lag_mcfs.append(rep_lag_mcf)
            rep_lag_mffs.append(rep_lag_mff)
            rep_lag_mfcs.append(rep_lag_mfc)

        # aggregate fitting results
        rep_crp = pd.concat(rep_crps).reset_index()
        rep_lag_mcf = pd.concat(rep_lag_mcfs).reset_index()
        rep_lag_mff = pd.concat(rep_lag_mffs).reset_index()
        rep_lag_mfc = pd.concat(rep_lag_mfcs).reset_index()

        # plot results
        sns.set(style='darkgrid')
        fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15, 25), sharey=True)

        # rep_crp
        sns.lineplot(ax=axes[0, 0], data=rep_crp[rep_crp['Position']=='First'].query(filt_neg), x='Lag', y="Recall Rate", 
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[0, 0], data=rep_crp[rep_crp['Position']=='First'].query(filt_pos), x='Lag', y="Recall Rate", 
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[0, 1], data=rep_crp[rep_crp['Position']=='Second'].query(filt_neg), x='Lag', y="Recall Rate", 
            hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(ax=axes[0, 1], data=rep_crp[rep_crp['Position']=='Second'].query(filt_pos), x='Lag', y="Recall Rate", 
            hue=varied_parameter, ci=None, legend=False)
        axes[0, 0].set_title('First Repetition Lag-Contiguity')
        axes[0, 1].set_title('Second Repetition Lag-Contiguity')
        
        # rep_lag_mcf
        sns.lineplot(
            ax=axes[1, 0], data=rep_lag_mcf[rep_lag_mcf['Position']=='First'].query(filt_neg), 
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[1, 0], data=rep_lag_mcf[rep_lag_mcf['Position']=='First'].query(filt_pos),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[1, 1], data=rep_lag_mcf[rep_lag_mcf['Position']=='Second'].query(filt_neg),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[1, 1], data=rep_lag_mcf[rep_lag_mcf['Position']=='Second'].query(filt_pos),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        axes[1, 0].set_title('MCF First Repetition Lag-Contiguity')
        axes[1, 1].set_title('MCF Second Repetition Lag-Contiguity')

        # rep_lag_mfc
        sns.lineplot(
            ax=axes[2, 0], data=rep_lag_mfc[rep_lag_mfc['Position']=='First'].query(filt_neg),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[2, 0], data=rep_lag_mfc[rep_lag_mfc['Position']=='First'].query(filt_pos),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[2, 1], data=rep_lag_mfc[rep_lag_mfc['Position']=='Second'].query(filt_neg),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[2, 1], data=rep_lag_mfc[rep_lag_mfc['Position']=='Second'].query(filt_pos),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        axes[2, 0].set_title('MFC First Repetition Lag-Contiguity')
        axes[2, 1].set_title('MFC Second Repetition Lag-Contiguity')

        # rep_lag_mff
        sns.lineplot(
            ax=axes[3, 0], data=rep_lag_mff[rep_lag_mff['Position']=='First'].query(filt_neg),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[3, 0], data=rep_lag_mff[rep_lag_mff['Position']=='First'].query(filt_pos),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[3, 1], data=rep_lag_mff[rep_lag_mff['Position']=='Second'].query(filt_neg),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        sns.lineplot(
            ax=axes[3, 1], data=rep_lag_mff[rep_lag_mff['Position']=='Second'].query(filt_pos),
            x='Lag', y="Recall Rate", hue=varied_parameter, ci=None, legend=False)
        axes[3, 0].set_title('MFF First Repetition Lag-Contiguity')
        axes[3, 1].set_title('MFF Second Repetition Lag-Contiguity')

        # render result
        plt.show()